In [1]:


import pandas as pd
 
# read by default 1st sheet of an excel file
dataframe1 = pd.read_excel('/content/Normalized_scores.xlsx')
 


In [2]:
import pandas as pd 

In [3]:
import pandas as pd 
import numpy as np
import copy
import matplotlib.pyplot as plt
from scipy.optimize import linprog 

In [4]:
oldframe = pd.DataFrame(columns = ['first', 'second', 'third'])

In [5]:
def solve_zerosum_with_linprog(U):
    '''solve_zerosum_with_linprog(): Solve a zero sum game using linear programming
    
        INPUT: U (k*k square matrix), payoffs in zero sum game (opponent gets -U.T)
        OUTPUT: alpha (k-vector) of probability weights for each action (the symmetric equilibrium)
    '''
    k, k2 = U.shape
    assert k == k2, f'Input matrix must be square, got {k}*{k2}'

    oo = np.zeros((1,k))
    ii = np.ones((1,k))

    # objective: c = [-1, 0, 0, ..., 0]
    c = np.insert(oo, 0, -1.0) # insert -1 in front (pos = index 0)
    
    # inequality constraints A*x <= b
    # top = [ 1 ...
    #         1 -1*A.Tl
    #         1  ...  ]
    # bot = [ 0 -1 0 0 
    #         0 0 -1 0 
    #         0 0 0 -1]
    top  = np.hstack( (ii.T, -1*U.T) )
    bot  = np.hstack( (oo.T, -1*np.eye(k)) )
    A_ub = np.vstack((top, bot))
    
    b_ub = np.zeros((1, 2*k))
    b_ub = np.matrix(b_ub)
    
    # contraints Ax = b
    # A = [0, 1, 1, ..., 1]
    A = np.matrix(np.hstack((0, np.ones((k,)))))
    b = 1.0 # just one condition so scalar 

    # v and alpha must be non-negative
    bounds = [(0,None) for i in range(k+1)]

    # call the solver
    sol = linprog(c=c, A_ub=A_ub, b_ub=b_ub, A_eq=A, b_eq=b)
    
    # remove the first element: just return the Nash EQ 
    alpha = sol.x[1:]
    return alpha


def best_response(U, i): 
    """best_response(): 
        INPUTS: 
            U: list of payoff matrices 
            i: (int) player for whom to do the best response 
        OUTPUT: 
            br: (NEQ*2) matrix, where br[:,0] is opponent strategies
                and br[:,1] are the best responses. If one strategy a
                has multiple best responses, then there will be several
                columns in br with br[:,0]==a. 
    """
    j = 1-i # opponent
    if i == 0: 
        Ui = U[0]
    elif i == 1: 
        Ui = U[1].T # so that i becomes row player 
    else: 
        raise Exception(f'Not implemented for n>2 players, got i={i}')

    nai, naj = Ui.shape

    # initialie 
    br = []

    for aj in range(naj):
        # column of U corresponding to the aj'th action of the opponent
        Ui_j = Ui[:, aj] 

        # find index values for the rows where Ui_j attains the max
        idim = 0 # there will not be more dimensions in our case 
        br_ij = np.where(Ui_j == Ui_j.max())[idim]

        for b in br_ij: 
            br.append([aj, b])

    return np.array(br)
    

def print_payoffs(U, A, round_decimals=None): 
    '''print_payoffs: Nicely formatted for a 2*2 game 
        INPUTS: 
            U1,U2: (matrices, dim=na1*na2) Payoffs 
            A1: (list of str, len=na1) List of actions of player 1
            A2: (list of str, len=na2) list of actions of player 2
            round_decimals: (int) Number of decimals of precision to print with 
        
        OUTPUT:
            tab: pandas dataframe, na1*na2 with payoff tuples 
    '''
    assert len(U) == 2, f'only implemented for 2-player games'
    assert len(A) == 2, f'only implemented for 2-player games'

    U1 = U[0]
    U2 = U[1]
    A1 = A[0]
    A2 = A[1]

    na1,na2 = U1.shape
    assert len(A1) == na1
    assert len(A2) == na2

    if not (round_decimals is None):
        assert np.isscalar(round_decimals), f'round_decimals must be an integer' 
        U1 = U1.round(round_decimals)
        U2 = U2.round(round_decimals)

    # "matrix" of tuples 
    X = [[(U1[r,c],U2[r,c]) for c in range(na2)] for r in range(na1)]

    # dataframe version 
    tab = pd.DataFrame(X, columns=A2, index=A1)
    return tab 

def find_undominated_actions(U_in, i, A, DOPRINT=False):
    '''find_undominated_actions: finds the actions for player i that are
        not strictly dominated by another action
        
        INPUTS: 
            U_in: (matrix, na1*na2) Payoffs (player 1, player 2)
            i: (integer) Which player we are currently examining
            A: (list) List of actions (len = # of actions for this player)
            
        OUTPUT: 
            AA: (list) undominated actions 
            IA: (list of integers) integers i s.t. AA = [A[i] for i in IA]
            ANYDOMINATED: (bool) True if at least one action was strictly dominated
    '''
    
    AA = []
    IA = []
    nA = len(A)
    
    # 1. ensure that U has actions of player i along 0th dimension 
    if i == 0: 
        # 1.a already the case 
        U = np.copy(U_in)
    else: 
        # 1.b transpose 
        U = U_in.T 
    
    # 2. determine if each action has other dominated actions 
    for ia in range(nA): 
        DOMINATED = False 
                
        for ia_ in range(nA): 
            # 2.a loop through all *other* strategies 
            if ia_ == ia: 
                continue

            # 2.b check if ia_ always gives a higher payoff than ia (i.e. domination)
            if np.all(U[ia_] > U[ia]): 
                DOMINATED = True
                break # exit search: enough that we have found one 
        
        # 2.c append or not 
        if not DOMINATED: 
            AA.append(A[ia])
            IA.append(ia)
            
    # 3. convenient boolean 
    ANYDOMINATED = (len(AA) < len(A))
    
    return AA,IA,ANYDOMINATED


def IESDS(A, U, DOPRINT=False, maxit=10000): 
    '''Iterated Elimination of Strictly Dominated Strategies 
        INPUTS: 
            A: (list of lists) n lists (one for each player), 
                    each has len = # of actions to player i
            U: (list, len=n) list of na1*na2 matrices of payoffs
            DOPRINT: (bool) whether to print output to terminal 
            maxit: (int) break algorithm if this count is ever reached
                (note: the algorithm is not approximate so we can compute 
                what maxit is in the worst case)
        OUTPUT: Actions and payoffs for the undominated game
            A_undominated: (n-list of vectors) 
            U_undominated: (n-list of matrices of payoffs)
    '''
    
    U_undominated = copy.copy(U)
    A_undominated = copy.copy(A)
    
    n = len(U)
    na1,na2 = U[0].shape

    # checks 
    assert n == 2, f'Code only implemented for 2-player games '
    assert len(A) == n
    for i in range(n): 
        assert len(A[i]) == U[i].shape[i]
        assert U[i].shape == (na1,na2), f'Payoff matrix for player {i+1} is {U[i].shape}, but {(na1,na2)} for player 1'

    # initialize flags 
    D = np.ones((n,), dtype='bool')
    
    for it in range(maxit): 

        for i in range(n): 
            # find undominated actions 
            A_undominated[i], IA, D[i] = find_undominated_actions(U_undominated[i], i, A_undominated[i], DOPRINT)

            # if we found at least one, remove it/them from the game 
            if D[i]: 
                # remove from both players' payoff matrices 
                for j in range(n): 
                    if i == 0: 
                        U_undominated[j] = U_undominated[j][IA, :]
                    else: 
                        U_undominated[j] = U_undominated[j][:, IA]


        # break once we have run an iteration without finding any strategies to remove 
        if D.any() == False: 
            break

    return A_undominated, U_undominated

In [6]:
pip install nashpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for nashpy: filename=nashpy-0.0.21-py3-none-any.whl size=15280 sha256=1838d8f67f76f6e0892a9b04e735ddd957366e9d4ef20b7f79f9de2b00d5ad1c
  Stored in directory: /root/.cache/pip/wheels/02/08/62/cf4fa931e0a317d180936b266169a57f4bb4eb801465bbe8a1
Successfully built nashpy


In [7]:
import pandas as pd 
import numpy as np 
import itertools
import nashpy


In [8]:
def compute_full_matrix(U1, U2, p, action_names=None): 
    '''
        Assumes that only player 2's type varies 
        (this means that player 1 has one action per row in U1, 
         while 2 has nA2**2 (one choice per type))
        Both players have one utility matrix for each realization 
        of player 2's type. 
         
        INPUTS: 
            U1: list of 2 payoff matrices for player 1 (row player)
            U2: list of 2 payoff matrices for player 2 (column player)
            p: (scalar) Probability that player 2 is the first type 
            action_names: [optional] 2-list of names of actions (nA1 and nA2 long)
        OUTPUTS: 
            t1, t2: wide-form payoff matrices suitable for finding the NE 
            A1, A2: names of actions 
    '''
    assert len(U1) == 2
    assert len(U2) == 2 
    assert np.isscalar(p)
    nA1, nA2 = U1[0].shape
    
    t1 = np.empty((nA1, nA2*nA2))
    t2 = np.empty((nA1, nA2*nA2))

    for ia1 in range(nA1): 
        i_col = 0 
        
        for a2_1 in range(nA2): 
            for a2_2 in range(nA2): 
                t1[ia1,i_col] = p * U1[0][ia1,a2_1] + (1.-p) * U1[1][ia1,a2_2]
                t2[ia1,i_col] = p * U2[0][ia1,a2_1] + (1.-p) * U2[1][ia1,a2_2]
                
                i_col += 1
    
    if action_names is None: 
        A1 = [f'{i}' for i in range(nA1)]
        A2 = [f'{a}{b}' for a in range(nA2) for b in range(nA2)]
    else: 
        assert len(action_names) == 2 
        A1 = action_names[0]
        assert len(A1) == nA1, f'Incorrect # of action names'
        a2 = action_names[1]
        assert len(a2) == nA2, f'Incorrect # of action names'
        
        A2 = [f'{a}{b}' for a in a2 for b in a2]
        
    return t1, t2, A1, A2
    


In [9]:
dataframe1.head(1)

,RP,RN,CP,CN
0,1.0,0.0,0.107955,0.011364


In [10]:
newframe = pd.DataFrame(columns = ['first review', 'second review', 'result'])

In [11]:
for i in range(0,10):
    for j in range(0,10):
        if i!=j:
            a= dataframe1.at[i, "CP"]#review1
            b= dataframe1.at[i, "CN"]
            c= dataframe1.at[i, "RP"]
            d= dataframe1.at[i, "RN"]
            # Review 1 payoffs
            u11  = np.array([[a,a], [b,b]])#context of R1
            u12  = np.array([[c,c], [d,d]])#rating of R1
            U1 = [u11, u12] # Player 1  ROW MATRIX 
            A1 = ['P', 'N']
         
            a1= dataframe1.at[j, "CP"]#review2
            b1= dataframe1.at[j, "CN"]
            c1= dataframe1.at[j, "RP"]
            d1= dataframe1.at[j, "RN"]
            # Review 2 payoffs
            u21  = np.array([[a1,a1], [b1,b1]])#context of R1
            u22  = np.array([[c1,c1], [d1,d1]])#rating of R1
            U2 = [u21, u22] # Player 1  ROW MATRIX 
            A1 = ['P', 'N']
         
            # Pr(player 2 is the PD type) writng probability of occurance
            if c1 != 5:
              p = c1/5
            else:
              p = 0.9   

            # player 1  with probablity (1-p)
            u1  = np.array([[a,a], [b,b]])#context of R1
            u2  = np.array([[c,c], [d,d]])#rating of R1
            U1 = [u1, u2] # Player 1  ROW MATRIX 
            A1 = ['P', 'N']

            # player 2 with probablity (p)
            u21 = np.array([[a1,b1],[a1,b1]]) # context of R2 
            u22 = np.array([[c1,d1],[c1,d1]]) # rating of R2
            U2 = [u21, u22]#PLAYER2 COLOUM MATRIX
            a2 = ['P', 'N']
            A2 = [f'{a}{b}' for a in a2 for b in a2]
         
            t1, t2, A1, A2 = compute_full_matrix(U1, U2, p, [A1, a2])
            A_, T_ = IESDS([A1, A2], [t1, t2], DOPRINT=True)

            print_payoffs([u1, u21], [A1, a2])
            print_payoffs([u2, u22], [A1, a2])
            result=[]
            data=[]
         
            result = print_payoffs(T_, A_, 3)
            print(i,j,result)
            newframe = newframe.append({'first review' : i, 'second review' : j, 'result' : result}, ignore_index = True)
           
            #result.append(i, result)
           # print_payoffs([t1, t2], [A1,  A2], 3)


0 1                NN
P  (0.964, 0.774)
0 2                NP
P  (0.857, 0.674)
0 3                PN
P  (0.929, 0.558)
0 4                PP
P  (0.822, 0.804)
0 5                NP
P  (0.857, 0.676)
0 6                PP
P  (0.822, 0.808)
0 7                PP
P  (0.822, 0.825)
0 8                PP
P  (0.822, 0.811)
0 9                PP
P  (0.822, 0.821)
1 0                PP
N  (0.668, 0.822)
1 2                NP
N  (0.694, 0.674)
1 3                PN
N  (0.747, 0.558)
1 4                PP
N  (0.668, 0.804)
1 5                NP
N  (0.694, 0.676)
1 6                PP
N  (0.668, 0.808)
1 7                PP
N  (0.668, 0.825)
1 8                PP
N  (0.668, 0.811)
1 9                PP
N  (0.668, 0.821)
2 0                PP
P  (0.641, 0.822)
2 1                NN
P  (0.768, 0.774)
2 3                PN
P  (0.737, 0.558)
2 4                PP
P  (0.641, 0.804)
2 5                NP
P  (0.673, 0.676)
2 6                PP
P  (0.641, 0.808)
2 7                PP
P  (0.641, 0.825)


In [12]:
newframe.to_excel("bayesian_output.xlsx") 

In [13]:
result.head()

,PP
P,"(0.821, 0.811)"


In [14]:
G = nashpy.Game(t1, t2)

eqs = list(G.support_enumeration())
print(f'Found {len(eqs)} equilibria')
for i,eq in enumerate(eqs): 
    print(f'{i+1}: s1 = {eq[0]}, s2 = {eq[1]}')

Found 1 equilibria
1: s1 = [1. 0.], s2 = [1. 0. 0. 0.]


In [15]:

result.to_excel("output.xlsx")